
### **Text Cell**

# 🧠 FDP Day 4 (Extended): Spanish Translation

Welcome\! In the last notebook, we built a model to translate English to French.

Now, we'll prove how flexible this model is by training it on a **new language (Spanish)** without changing the model's structure at all. This demonstrates a core concept in Deep Learning: the same architecture can learn different tasks *if* it's given different data.

## 📝 Our Plan

1.  **Change Data Source:** We'll download the English-Spanish dataset instead of the English-French one.
2.  **Inspect Data:** We'll add new steps to look at our raw data and understand our new "vocabulary."
3.  **Tokenize:** We'll create our character-to-index maps for English and **Spanish**.
4.  **Vectorize:** We'll one-hot encode our data (this code is identical).
5.  **Build Model:** We'll use the *exact same* Encoder-Decoder model as before.
6.  **Train Model:** We'll train the model on the Spanish sentences.
7.  **Run Inference:** We'll create the inference models (this code is identical).
8.  **Test:** We'll write a **new, simple function** that lets you type any English sentence and get a Spanish translation\!

-----

### **Text Cell**

## Cell 1: Setup - Importing Libraries

No changes here. We still need TensorFlow, Keras, and Numpy.

-----

### **Code Cell**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, GRU, Dense
from tensorflow.keras.models import Model
import numpy as np
import os

print("Libraries imported.")

Libraries imported.


-----

### **Text Cell**

## Cell 2: Define Dataset Parameters

These parameters are good starting points, so we'll keep them. We'll train on the first `10,000` samples. For a real-world model, you would want *much* more data (the full dataset has over 140,000 sentences\!) and train for more epochs.

*Note:* Training for 100 epochs will take some time. For a quick test, you can set `EPOCHS = 20`.

-----

### **Code Cell**

In [2]:
# --- Parameters ---
BATCH_SIZE = 64
EPOCHS = 100
LATENT_DIM = 256  # This is the "size" of the thought vector
NUM_SAMPLES = 10000 # Number of sentences to train on

# --- Data Path ---
# This will be our new data file!
DATA_PATH = "spa.txt"

-----

### **Text Cell**

## Cell 3: Download and Unzip the **Spanish** Data

This is our first major change\! Instead of `fra-eng.zip`, we are downloading `spa-eng.zip`. The `manythings.org` website provides many language pairs in this exact same format, making it easy to swap.

-----

### **Code Cell**

In [3]:
# Download the data
!curl -O http://www.manythings.org/anki/spa-eng.zip
!unzip -o spa-eng.zip

print("Spanish data downloaded and unzipped.")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5338k  100 5338k    0     0  9245k      0 --:--:-- --:--:-- --:--:-- 9251k
Archive:  spa-eng.zip
  inflating: _about.txt              
  inflating: spa.txt                 
Spanish data downloaded and unzipped.


-----

### **Text Cell**

## Cell 4: Parse the Data

The parsing logic is identical. We read the file line by line. Each line is:
`English Sentence` \<tab\> `Spanish Sentence` \<tab\> `Attribution`

We'll store the English in `input_texts` and the Spanish in `target_texts`.

We still add `\t` (tab) as the "start" token and `\n` (newline) as the "end" token to our target (Spanish) sentences. This is crucial for the decoder to learn *when to start* and *when to stop* generating text.

-----

### **Code Cell**

In [4]:
# --- Parse the data --
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# Read the file
with open(DATA_PATH, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

# Process each line
for line in lines[: min(NUM_SAMPLES, len(lines) - 1)]:
    # Split the line into English (input) and Spanish (target)
    try:
        input_text, target_text, _ = line.split("\t")
    except:
        print(f"Skipping bad line: {line}")
        continue

    # We use "tab" as the "start sequence" token
    # and "\n" as the "end sequence" token for the target.
    target_text = "\t" + target_text + "\n"

    input_texts.append(input_text)
    target_texts.append(target_text)

    # Add any new characters to our "vocabulary" sets
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

print(f"Loaded {len(input_texts)} sentence pairs.")

Loaded 10000 sentence pairs.


-----

### **Text Cell**

## Cell 5: **New Cell** - Inspect the Data

Let's add a step to see what we've loaded. This is always a good idea. We'll print the first 10 sentence pairs. Notice the `\t` and `\n` in the Spanish sentences.

-----

### **Code Cell**

In [5]:
print("--- 10 Sample Pairs ---")
for i in range(10):
    print(f"Input:    {input_texts[i]}")
    # We use repr() to show the "hidden" \t and \n characters
    print(f"Target:   {repr(target_texts[i])}")
    print("---")

--- 10 Sample Pairs ---
Input:    Go.
Target:   '\tVe.\n'
---
Input:    Go.
Target:   '\tVete.\n'
---
Input:    Go.
Target:   '\tVaya.\n'
---
Input:    Go.
Target:   '\tVáyase.\n'
---
Input:    Hi.
Target:   '\tHola.\n'
---
Input:    Run!
Target:   '\t¡Corre!\n'
---
Input:    Run!
Target:   '\t¡Corran!\n'
---
Input:    Run!
Target:   '\t¡Huye!\n'
---
Input:    Run!
Target:   '\t¡Corra!\n'
---
Input:    Run!
Target:   '\t¡Corred!\n'
---


-----

### **Text Cell**

## Cell 6: Create the Character Vocabularies

Now we find all the unique characters in our dataset. This is our "vocabulary."

Why use **character-level** tokenization?

  * **Small Vocabulary:** Our model only needs to learn \~70-100 tokens (characters). A *word-level* model would need to learn 50,000+ unique words, which is much bigger.
  * **Handles Typos & New Words:** A character model can spell out any word, even ones it hasn't seen (like "Googol"). A word model would see this as an "Unknown" token.
  * **The Downside:** It's slower and has to learn grammar and spelling from scratch.

-----

### **Code Cell**

In [6]:
# Sort the vocabularies
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# Get the size of our vocabularies and longest sentences
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("\n--- Data Stats ---")
print("Number of unique input tokens (English characters):", num_encoder_tokens)
print("Number of unique output tokens (Spanish characters):", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


--- Data Stats ---
Number of unique input tokens (English characters): 68
Number of unique output tokens (Spanish characters): 84
Max sequence length for inputs: 16
Max sequence length for outputs: 47


-----

### **Text Cell**

## Cell 7: Create Character-to-Index Maps

Models don't read letters. They read numbers. This step creates a simple dictionary (a "lookup map") that converts each character to a unique number.

`'a'` -\> `10`
`'b'` -\> `11`
etc.

We'll print the first 15 mappings for both languages. Notice that the Spanish vocabulary (`target_token_index`) includes special characters like `¿`, `¡`, and `ñ` that the English one doesn't.

-----

### **Code Cell**

In [7]:
# Create the character-to-index lookup maps
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

print("--- Input Token Index (English) ---")
print(list(input_token_index.items())[:15])

print("\n--- Target Token Index (Spanish) ---")
print(list(target_token_index.items())[:15])
# Note: You should see special Spanish characters in this list!

--- Input Token Index (English) ---
[(' ', 0), ('!', 1), ('$', 2), ('&', 3), ("'", 4), (',', 5), ('.', 6), ('0', 7), ('1', 8), ('2', 9), ('3', 10), ('5', 11), ('6', 12), ('7', 13), ('8', 14)]

--- Target Token Index (Spanish) ---
[('\t', 0), ('\n', 1), (' ', 2), ('!', 3), ('"', 4), ('&', 5), ("'", 6), (',', 7), ('-', 8), ('.', 9), ('0', 10), ('1', 11), ('2', 12), ('3', 13), ('5', 14)]


-----

### **Text Cell**

## Cell 8: Vectorization (One-Hot Encoding)

This is the most important pre-processing step. We create three massive 3D arrays to hold our data.

A 3D array (or "Tensor") has a shape:
`(samples, timesteps, features)`

  * **samples:** The number of sentences (10,000).
  * **timesteps:** The maximum length of a sentence (e.g., `max_encoder_seq_length`).
  * **features:** The size of our vocabulary (e.g., `num_encoder_tokens`).

We are using **One-Hot Encoding**. This means for each character, we create a vector that is all `0`s *except* for a single `1` at the index of that character.

If `'H'` is index `35`, the vector for `'H'` would be:
`[0, 0, 0, ... 0, 1, 0, ... 0]` (with the `1` at position 35).

-----

### **Text Cell**

## Cell 9: Understanding "Teacher Forcing"

You'll see we create *two* target arrays: `decoder_input_data` and `decoder_target_data`. This is for a technique called **Teacher Forcing**.

The model needs to learn: "Given the *previous* character, predict the *next* one."

  * **`decoder_input_data`** is the "question." It's the Spanish sentence starting with `\t`.
  * **`decoder_target_data`** is the "answer." It's the *same sentence* shifted one step forward.

**Example:**

  * **Input Sentence:** `Hola`
  * `decoder_input_data` will be: `\t`, `H`, `o`, `l`, `a`
  * `decoder_target_data` will be: `H`, `o`, `l`, `a`, `\n`

The model learns:

1.  When you see `\t`, predict `H`.
2.  When you see `H`, predict `o`.
3.  When you see `o`, predict `l`.
    ...and so on.

-----

### **Code Cell**

In [8]:
# --- Create the empty numpy arrays ---
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

print("Created empty data arrays...")

# --- Fill the arrays with one-hot vectors ---
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # Fill encoder_input_data
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    # Pad the rest of the sequence with spaces
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

    # Fill decoder_input_data and decoder_target_data
    for t, char in enumerate(target_text):
        # decoder_input_data is the sequence itself
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data is the sequence shifted by one
            # (it does not include the start token '\t')
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    # Pad the rest of the sequences with spaces
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

print("Data vectorization (one-hot encoding) complete.")
print(f"Shape of encoder_input_data: {encoder_input_data.shape}")
print(f"Shape of decoder_input_data: {decoder_input_data.shape}")
print(f"Shape of decoder_target_data: {decoder_target_data.shape}")

Created empty data arrays...
Data vectorization (one-hot encoding) complete.
Shape of encoder_input_data: (10000, 16, 68)
Shape of decoder_input_data: (10000, 47, 84)
Shape of decoder_target_data: (10000, 47, 84)


-----

### **Text Cell**

## Cell 10: Define the Encoder

This part is **identical** to the French notebook. Its job is to read the *entire* English sentence, one character at a time, and compress its meaning into a single vector of size `LATENT_DIM` (256).

This "thought vector" (or "context vector") is `state_h`. It's the only thing we pass to the decoder.

(We include `reset_after=False` to fix a known TensorFlow bug with GRU/LSTM layers).

-----

### **Code Cell**

In [9]:
# --- Encoder ---
# Define the input layer
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# Define the GRU layer
# We set return_state=True to get the final "thought vector"
encoder_gru = GRU(LATENT_DIM, return_state=True, reset_after=False)

# Run the inputs through the GRU
encoder_outputs, state_h = encoder_gru(encoder_inputs)

# We discard the `encoder_outputs` and only keep the state (`state_h`)
encoder_states = [state_h]

print("Encoder built.")

Encoder built.


-----

### **Text Cell**

## Cell 11: Define the Decoder

This is also **identical**. Its job is to generate the Spanish sentence.

It's more complex than the encoder:

1.  It takes the `decoder_inputs` (the "teacher-forced" Spanish sentence).
2.  It takes the `encoder_states` (the "thought vector" from the encoder) as its **`initial_state`**. This is the magic link\! This is how the decoder knows *what* to translate.
3.  It `return_sequences=True` because it needs to output a prediction at *every* time step (for every character).
4.  The final `Dense` layer with `softmax` is a classifier. It looks at the GRU's output and predicts "which character in the Spanish vocabulary is most likely to be next?"

-----

### **Code Cell**

In [10]:
# --- Decoder ---
# Define the input layer
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Define the GRU layer
decoder_gru = GRU(LATENT_DIM, return_sequences=True, return_state=True, reset_after=False)

# Run the inputs through the GRU, using the ENCODER'S state as the STARTING state
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)

# A Dense layer to act as our character-classifier
decoder_dense = Dense(num_decoder_tokens, activation="softmax")

# Apply the classifier to the GRU's outputs
decoder_outputs = decoder_dense(decoder_outputs)

print("Decoder built.")

Decoder built.


-----

### **Text Cell**

## Cell 12: Define and Compile the **Training** Model

This is the final model we will *train*. It connects the inputs and outputs we just defined.

  * **Inputs:** `encoder_inputs` (English) and `decoder_inputs` (Spanish, for teacher-forcing).
  * **Outputs:** `decoder_outputs` (the model's predictions).

We compile it with `categorical_crossentropy` because this is a classification problem (choosing one character from many).

-----

### **Code Cell**

In [11]:
# --- Full Model ---
# This is the model we will *train*
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

print("Model compiled.")
model.summary()

Model compiled.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 68)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 84)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ [(None, 256),     │    249,600 │ input_layer[0][0] │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ [(None, None,     │    261,888 │ input_layer_1[0]… │
│                     │ 256), (None,      │            │ gru[0][1]         │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 84)  │     21,588 │ gru_1[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 533,076 (2.03 MB)

 Trainable params: 533,076 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

-----

### **Text Cell**

## Cell 13: Train the Model

This is the moment of truth. We `fit` the model, passing our two inputs and one target.

This will take a few minutes. If you are using a GPU in Colab (go to `Runtime > Change runtime type > T4 GPU`), it will be much faster.

-----

### **Code Cell**

In [12]:
print("Starting model training...")

model.fit(
    [encoder_input_data, decoder_input_data], # Pass both inputs
    decoder_target_data,                      # Pass the target
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    verbose=1
)

print("\nTraining complete.")
# Save the model
model.save("seq2seq_spanish_translator.keras")
print("Model saved.")

Starting model training...
Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.6701 - loss: 1.7492 - val_accuracy: 0.6579 - val_loss: 1.2217
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7252 - loss: 1.0219 - val_accuracy: 0.7168 - val_loss: 0.9921
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7605 - loss: 0.8532 - val_accuracy: 0.7365 - val_loss: 0.8984
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7812 - loss: 0.7500 - val_accuracy: 0.7528 - val_loss: 0.8267
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7909 - loss: 0.7098 - val_accuracy: 0.7586 - val_loss: 0.8009
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7931 - loss: 0.6889 - val_accuracy: 0.7677 - val_loss: 0.7799
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7986 - loss: 0.6691 - val_accuracy: 0.7697 - val_loss: 0.7615
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accurac

-----

### **Text Cell**

## Cell 14: The Problem with Our Training Model

**STOP\!** Our training is done, but we have a problem.

The model we just trained (`model`) is **unusable for real translation**.

Why? Because it was built with *Teacher Forcing*. It **requires** you to feed it the correct Spanish translation in `decoder_inputs` to work. But if we already *had* the Spanish translation, we wouldn't need a model\!

We need a new setup for *inference* (translation). We will do this by re-using the *layers* we just trained, but connecting them in a new way.

We need two new models:

1.  **`encoder_model`**: Takes an English sentence and just gives us the "thought vector" (`encoder_states`).
2.  **`decoder_model`**: Takes one character at a time (plus the previous thought vector) and predicts the *next* character and the *new* thought vector.

-----

### **Text Cell**

## Cell 15: Create the Encoder Inference Model

This is simple. We define a new `Model` that takes the *original* `encoder_inputs` and outputs the *original* `encoder_states`. This model's only job is to create the "thought vector."

-----

### **Code Cell**

In [13]:
# We create a new model that just runs the encoder
encoder_model = Model(encoder_inputs, encoder_states)

print("Encoder (Inference) Model built.")
encoder_model.summary()

Encoder (Inference) Model built.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, 68)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ [(None, 256), (None,   │       249,600 │
│                                 │ 256)]                  │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 249,600 (975.00 KB)

 Trainable params: 249,600 (975.00 KB)

 Non-trainable params: 0 (0.00 B)

-----

### **Text Cell**

## Cell 16: Create the Decoder Inference Model

This is the trickiest part. We have to re-build the decoder manually.

This model will be run in a loop.

1.  **Input:** It needs the *last predicted character* (as a one-hot vector) AND the *previous state vector* from the encoder (or the last loop).
2.  **Output:** It will give us the *prediction* for the next character (a softmax probability vector) AND the *new state vector* to be used in the next loop.

-----

### **Code Cell**

In [14]:
# --- Decoder (Inference) ---
# We need to define new inputs for the state, as it will be fed in a loop
decoder_state_input_h = Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_state_input_h]

# We need a new input for the single character
# Shape: (batch_size, 1, num_decoder_tokens) - we use 1 for the timestep
decoder_inputs_single_step = Input(shape=(1, num_decoder_tokens))

# We use the *original* GRU layer, but call it on our new inputs
# We get the outputs and the new state
decoder_outputs, state_h_dec = decoder_gru(
    decoder_inputs_single_step, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec]

# We use the *original* dense layer
decoder_outputs = decoder_dense(decoder_outputs)

# This is our final inference-mode decoder
decoder_model = Model(
    [decoder_inputs_single_step] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

print("Decoder (Inference) Model built.")
decoder_model.summary()

Decoder (Inference) Model built.


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 1, 84)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ [(None, 1, 256),  │    261,888 │ input_layer_3[0]… │
│                     │ (None, 256)]      │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1, 84)     │     21,588 │ gru_1[1][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 283,476 (1.08 MB)

 Trainable params: 283,476 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

-----

### **Text Cell**

## Cell 17: Create the `decode_sequence` Function

This function ties everything together. It contains the "inference loop" logic.

I have added detailed comments to explain each step.

-----

### **Code Cell**

In [15]:
# --- Setup reverse lookup ---
# We need to go from index-back-to-character to read the output
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

print("Reverse lookup dictionaries created.")


def decode_sequence(input_seq):
    # 1. ENCODE: Run the input sentence through the encoder
    # This gives us the "thought vector" (states_value)
    states_value = encoder_model.predict(input_seq, verbose=0)

    # 2. START: Create a 1-character-long "target" sequence
    # This character is the "start" token ('\t')
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # 3. LOOP: Start our translation loop
    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        # 4. PREDICT: Feed the current token and the state into the decoder
        output_tokens, h = decoder_model.predict([target_seq] + [states_value], verbose=0)

        # 5. SAMPLE: Find the most likely next character (the one with the highest prob)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]

        # 6. APPEND: Add the new character to our sentence
        decoded_sentence += sampled_char

        # 7. CHECK: Stop if we hit the "end" token ('\n') or the max length
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # 8. UPDATE: Prepare for the next loop
        # The new "current token" is the character we just predicted
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # The new "state" is the state `h` that the decoder just returned
        states_value = h

    return decoded_sentence

print("\nDecode function created.")

Reverse lookup dictionaries created.

Decode function created.


-----

### **Text Cell**

## Cell 18: Test on Training Samples

Let's see how our model did. We'll run it on 20 random samples from the training set.

You can see that for simple sentences, it learns quite well\! For more complex ones, it might struggle. This is because we only used 10,000 samples.

-----

### **Code Cell**

In [16]:
# --- Test the model on 20 samples ---
for seq_index in range(0, 200, 10): # Take 20 samples from the set
    # Take one sequence
    input_seq = encoder_input_data[seq_index : seq_index + 1]

    # Run our function
    decoded_sentence = decode_sequence(input_seq)

    print("-" * 30)
    print(f"Sample #{seq_index}")
    print(f"Input:    {input_texts[seq_index]}")
    # .strip() removes the \t and \n for clean viewing
    print(f"Actual:   {target_texts[seq_index].strip()}")
    print(f"Predicted: {decoded_sentence.strip()}")

------------------------------
Sample #0
Input:    Go.
Actual:   Ve.
Predicted: Vete.
------------------------------
Sample #10
Input:    Run.
Actual:   Corra.
Predicted: Prueba un poco.
------------------------------
Sample #20
Input:    Fire!
Actual:   ¡Incendio!
Predicted: ¡De perdudo!
------------------------------
Sample #30
Input:    Stay.
Actual:   Quédese.
Predicted: Quédate.
------------------------------
Sample #40
Input:    Wait.
Actual:   Espera.
Predicted: Esperen.
------------------------------
Sample #50
Input:    Hurry!
Actual:   ¡Daos prisa!
Predicted: ¡Aquí esto!
------------------------------
Sample #60
Input:    I won!
Actual:   ¡He ganado yo!
Predicted: ¡He ganado!
------------------------------
Sample #70
Input:    Shoot!
Actual:   ¡Dispare!
Predicted: ¡Disparen!
------------------------------
Sample #80
Input:    Eat it.
Actual:   Cómetelo.
Predicted: Espéjadlo.
------------------------------
Sample #90
Input:    Go now.
Actual:   Ve ahora mismo.
Predicted: Ve ya

-----

### **Text Cell**

## Cell 19: **New Cell** - Test Your Own Sentences\!

This is the best part. Why just test on old data? Let's write a function that lets you type a *new* English sentence and see the translation.

This function will:

1.  Take your string (e.g., "How are you?").
2.  Convert it to the one-hot encoded `(1, max_seq_len, num_tokens)` array that the `encoder_model` expects.
3.  Call our `decode_sequence` function.
4.  Print the result\!

**Note:** The model will only know characters that were in its original vocabulary. If you use a character it hasn't seen (like `*` or `#`), it will fail.

-----

### **Code Cell**

In [17]:
def translate_sentence(input_text):
    print(f"Translating: '{input_text}'")

    # 1. Create the empty input array
    input_seq = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32"
    )

    # 2. Fill it with one-hot vectors for the input text
    for t, char in enumerate(input_text):
        if char in input_token_index:
            input_seq[0, t, input_token_index[char]] = 1.0
        else:
            print(f"Warning: Unknown character '{char}' skipped.")

    # Pad the rest with spaces
    input_seq[0, t + 1 :, input_token_index[" "]] = 1.0

    # 3. Decode the sequence
    decoded_sentence = decode_sequence(input_seq)

    print(f"Spanish:   {decoded_sentence.strip()}")


# --- Try it out! ---
translate_sentence("How are you?")
print("-" * 30)
translate_sentence("I love you.")
print("-" * 30)
translate_sentence("Stop!")
print("-" * 30)
translate_sentence("Run.")

Translating: 'How are you?'
Spanish:   ¿Cómo está tuenta?
------------------------------
Translating: 'I love you.'
Spanish:   Te entiendo.
------------------------------
Translating: 'Stop!'
Spanish:   ¡Parad!
------------------------------
Translating: 'Run.'
Spanish:   Prueba un poco.
